In [ ]:
#@title Mount Google Drive and Change Directory to NEUTRINO
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/My Drive/NEUTRINO

In [ ]:
#@title Change permission
%%bash
find . -type f -exec chmod 755 \{\} \;

In [ ]:
#@title Load a musicxml file (only if lyrics is written in Romaji)

file_path_and_name = '' #@param {type:"string"}

In [ ]:
#@title Load dictionary data and convert the lyrics into hiragana
import os
import re
dic = {}
with open('/content/drive/MyDrive/NEUTRINO/dic.txt') as f:
    for line in f.readlines():
        data = line.split()
        dic[data[0]]=data[1]


f = open(file_path_and_name)
print(sum([1 for _ in open(file_path_and_name)]))
data = f.readlines()

new_lines = ''
for i, line in enumerate(data):
  if 'text' in line:
    romaji = re.search(r'<text>(.+)</text>', line).group(1) #doesn't contain words 
    #print(i+1, romaji)
    for k, v in dic.items():
      if romaji.lower()==k:
        #print('this is old version: ' + line)
        hiragana = romaji.replace(romaji, v)
        line = '<text>'+hiragana+'</text>'
        #print('this is new version: ' + line)
  new_lines += line
f.close()

In [ ]:
#@title The name and directory of a new musicxml
new_file_name_and_location = '' #@param {type:"string"}

In [ ]:
#@title Save the new musicxml file
nmusicxml = open(new_file_name_and_location, "w")
nmusicxml.write(new_lines)
nmusicxml.close()

In [ ]:
#@title Run NEUTRINO
%%bash
# Project settings
BASENAME=Basename Here #@param (type:"string")
NumThreads=0

# musicXML_to_label
SUFFIX=musicxml

# NEUTRINO
ModelDir=SEVEN #@param (type:"string")
StyleShift=0

# WORLD
PitchShift=1.0
FormantShift=1.0

# WORLD(beta version option)
SmoothPitch=0.0
SmoothFormant=0.0
EnhanceBreathiness=0.0

# PATH to currennt library
export LD_LIBRARY_PATH=$PWD/bin:$PWD/NSF/bin:$LD_LIBRARY_PATH

echo "`date +"%M:%S.%2N"` : start MusicXMLtoLabel"
./bin/musicXMLtoLabel score/musicxml/${BASENAME}.${SUFFIX} score/label/full/${BASENAME}.lab score/label/mono/${BASENAME}.lab

echo "`date +"%M:%S.%2N"` : start NEUTRINO"
./bin/NEUTRINO score/label/full/${BASENAME}.lab score/label/timing/${BASENAME}.lab ./output/${BASENAME}.f0 ./output/${BASENAME}.mgc ./output/${BASENAME}.bap ./model/${ModelDir}/ -n ${NumThreads} -k ${StyleShift} -t -m

echo "`date +"%M:%S.%2N"` : start WORLD"
./bin/WORLD output/${BASENAME}.f0 output/${BASENAME}.mgc output/${BASENAME}.bap -f ${PitchShift} -m ${FormantShift} -p ${SmoothPitch} -c ${SmoothFormant} -b ${EnhanceBreathiness} -o output/${BASENAME}_syn.wav -n ${NumThreads} -t

echo "`date +"%M:%S.%2N"` : start NSF"
./bin/NSF_IO score/label/full/${BASENAME}.lab score/label/timing/${BASENAME}.lab output/${BASENAME}.f0 output/${BASENAME}.mgc output/${BASENAME}.bap ${ModelDir} output/${BASENAME}_nsf.wav -t

echo "`date +"%M:%S.%2N"` : END"